## Библиотеки 

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## Функции

In [2]:
def apk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

## Получение данных

In [3]:
df_train = pd.read_csv('df_train.csv', sep=';')
df_test = pd.read_csv('df_test.csv', sep=';')

In [4]:
df_train['Data'] = df_train.Data.apply(lambda s: list(map(int, s.split(','))))
df_train['Target'] = df_train.Target.apply(lambda s: list(map(int, s.split(','))))
df_test['Data'] = df_test.Data.apply(lambda s: list(map(int, s.split(','))))

## Модель 1: ранжирование по вероятностям
В модели для ранжирования кодов я учитываю временной компонент, то есть более новым транзакциям в последовательности даю больший вес за счет учета их позиций. К примеру, для входной последовательности `[1, 1, 1, 1, 1, 2, 2, 2]` выходная последовательность будет начинаться на `[2, 1 ...]`, поскольку сумма позиций для двойки (6+7+8=21) будет больше, чем для единицы (1+2+3+4+5=15)

In [5]:
# Определяю общий топ-10 кодов по их доле во всем корпусе транзакций
total_top_10 = (df_train['Data'] + df_train['Target'] + df_test['Data']).explode().value_counts(ascending=False).index[:10]
# Длинные последовательности транзакций я буду использовать не целиком,
# поэтому определю максимальную длину исходя из среднего значения и подобранного коэффициента
alpha = 1.9
cutoff = int(np.mean([len(x) for x in df_test['Data']]) / alpha)

In [6]:
def tops_by_proba(seq, cutoff=cutoff, drop_from=5):
    seq = seq[-cutoff:]
    seq_len = len(seq)
    # Считаю сумму всех индексов в последовательности кодов, индексация в этом случае начинается с 1
    pos_sum = seq_len * (seq_len+1) / 2
    probas = {}

    for code in np.unique(seq):
        # Определяю индексы(позиции), на которых код встречается в последовательности
        positions = np.where(np.array(seq) == code)[0] + 1
        if len(positions) >= drop_from:
            # Вероятность кода для последовательности определяю как отношение суммы его позиций к сумме всех индексов
            probas[code] = sum(positions) / pos_sum
    
    output = sorted(probas, key=probas.get, reverse=True)
    
    if len(output) < 10:
        # Добавляю коды из общего топ-10, если выходная последовательность получилась меньше 10
        output += [x for x in total_top_10 if x not in output]

    return output[:10]

In [7]:
df_train['tops_by_proba'] = df_train['Data'].apply(tops_by_proba, drop_from=2)
print(mapk(df_train['Target'], df_train['tops_by_proba']))

0.336321564551694


In [8]:
df_test['Predicted'] = df_test['Data'].apply(tops_by_proba, drop_from=2)

submission = df_test[['Id', 'Predicted']]
submission['Predicted'] = submission['Predicted'].apply(lambda x: str(x).replace(',', '')[1:-1])
submission.to_csv('submission_tops_by_proba_final_2.csv', index=False)

## Модель 2: строю выходную последовательность по вероятностям, избегая по возможности повторение кодов
Почему я это избегаю? Потому что подстраиваться необходимо под метрику MAP@K, которая не засчитывает повторения. Однако в этой модели я не учитываю топ-10 кодов из всего набора данных, поэтому для некоторых клиентов повторений будет не избежать.

Почему здесь я пытаюсь построить последовательность а не ранжирую коды? Потому что таргет по сути является последовательностью, а не набором уникальных кодов. Эта модель - попытка найти компромисс, построив последовательность, но при этом не завалиться на метрике MAP@K.

In [9]:
def seq_by_weighted_proba_no_repeats(seq):
    seq_len = len(seq)
    # Считаю сумму всех индексов в последовательности кодов, индексация в этом случае начинается с 1
    pos_sum = seq_len * (seq_len+1) / 2
    probas = pd.Series(index=np.unique(seq))

    for code in probas.index:
        # Определяю индексы(позиции), на которых код встречается в последовательности
        positions = np.where(np.array(seq) == code)[0] + 1
        # Вероятность кода для последовательности определяю как отношение суммы его позиций к сумме всех индексов
        probas[code] = sum(positions) / pos_sum
    # Для построения выходной последовательности считаю количество повторений кодов исходя из их вероятностей
    counts = round(probas.sort_values(ascending=False) * 10)
    # Заношу в отдельный список коды с около-нулевой вероятностью
    outsider_codes = list(counts[counts == 0].index)
    output = [None]

    while counts.sum() != 0:
        # Итерируюсь по отсортированному списку кодов, чтобы по возможности избежать повторений
        for code in counts.index:
            if counts[code] != 0:
                # Заношу в выходную последовательность код, если ему не предшествует тот же код, или если кончились аутсайдеры
                if code != output[-1] or not outsider_codes:
                    output.append(code)
                    # Для занесенного кода вычитаю 1 из количества его повторений
                    counts[code] -= 1
                else:
                    # Если код "хочет" повториться, но есть аутсайдеры - вынимаю из списка самого вероятного аутсайдера
                    output.append(outsider_codes.pop(0))
    # Получившуюся последовательность дополняю оставшимися аутсайдерами
    output += outsider_codes
    # Если длина последовательности меньше 10 (без учета None в начале) - дополняю ее самым вероятным кодом до длины 10
    output += [counts.index[0]] * (11 - len(output))

    return output[1:11]

In [10]:
df_train['seq_by_weighted_proba_no_repeats'] = df_train['Data'].apply(seq_by_weighted_proba_no_repeats)
print(mapk(df_train['Target'], df_train['seq_by_weighted_proba_no_repeats']))

0.3041419814525175


In [11]:
df_test['Predicted'] = df_test['Data'].apply(seq_by_weighted_proba_no_repeats)

submission = df_test[['Id', 'Predicted']]
submission['Predicted'] = submission['Predicted'].apply(lambda x: str(x).replace(',', '')[1:-1])
submission.to_csv('submission_seq_by_weighted_proba_no_repeats_final.csv', index=False)